In [4]:
import polars as pl
from pathlib import Path

# stats generated from ~1300 replays from 1/1/2023 - 4/30/2023
l_cancel = pl.read_parquet(Path(R"./Output/l_cancels.parquet"))
l_cancel.schema

{'date_time': Datetime(time_unit='us', time_zone='America/Chicago'),
 'slippi_version': Utf8,
 'match_id': Utf8,
 'match_type': Utf8,
 'game_number': Int64,
 'stage': Utf8,
 'duration': Duration(time_unit='ms'),
 'result': Utf8,
 'port': Utf8,
 'connect_code': Utf8,
 'character': Utf8,
 'costume': Utf8,
 'opnt_character': Utf8,
 'frame_index': Int64,
 'stocks_remaining': Int64,
 'l_cancel': Boolean,
 'trigger_input_frame': Int64,
 'during_hitlag': Boolean,
 'move': Utf8,
 'position': Utf8,
 'fastfall': Boolean}

In [5]:
# Total l cancels
l_cancel.filter(pl.col("character") == "FALCO").groupby("l_cancel").agg(pl.count())

l_cancel,count
bool,u32
false,3073
true,19673


That's 22,746 aerials total, with an l cancel percentage of ~86.49%

In [6]:
# L cancel percent by character
l_cancel.groupby("character").agg(pl.col("l_cancel").mean())

character,l_cancel
str,f64
"""FOX""",0.807692
"""MARTH""",0.819138
"""FALCO""",0.864899
"""CAPTAIN_FALCON…",0.809929


I guess my secondaries are all pretty sloppy. Particularly embarassing because I pull out the secondary Marth in tournament, and fox a meme character that I hardly play even in friendlies

In [7]:
# L cancel percentage by move
l_cancel.filter(pl.col("character") == "FALCO").groupby(pl.col("move")).agg([pl.col("l_cancel").mean()])

move,l_cancel
str,f64
"""NAIR""",0.903505
"""FAIR""",0.817797
"""DAIR""",0.873635
"""BAIR""",0.787123
"""UAIR""",0.865533


Back to falco only, we can check percentage by move. Bair is probably as low as it is from missed AC bairs and drop cancel bairs. It may be worth learning to l cancel that, but it's hardly necessary since it's inconsistent and usually unexpected.

Nair vs dair is interesting, but I wonder how much of it is during combos vs during shield pressure. Might be worth looking into for a future stat type. 

Uair and Fair i've started using a lot more in combos, but the multihit makes timing l cancels tricky (especially on SHFFL uair). This is definitely grind-able solo though, so I'll see where this stat is in a month or two after some grinding.

In [8]:
(  # l cancel percentage based on landing location
    l_cancel.filter(pl.col("character") == "FALCO")
    .groupby(pl.col("position"))
    .agg([pl.col("l_cancel").mean(), pl.count()])
)


position,l_cancel,count
str,f64,u32
"""RIGHT_PLATFORM…",0.703523,1022
"""MAIN_STAGE""",0.892154,19769
"""TOP_PLATFORM""",0.672666,889
"""LEFT_PLATFORM""",0.676749,1058
"""RANDALL""",0.375,8


Pretty much what you'd expect, my l cancel percentage dips pretty heavily when landing on platforms. A small portion can be attributed to drop cancel bair, but I know most of this is due to slop. I've been incorporating platform play much more often, and this sort of thing really slows me down when I need to keep the pace up. It's not as easy to solo grind this, but I think I know which scenarios I consistently miss l cancels in, so I should be able to work on this.

In [16]:
(  # L cancel percent by opponent character, sorted by number of l cancel occurances
    l_cancel.filter(pl.col("character") == "FALCO")
    .groupby(pl.col("opnt_character"))
    .agg([pl.col("l_cancel").mean(), pl.count()])
    .sort(pl.col("count"), descending=True)
)


opnt_character,l_cancel,count
str,f64,u32
"""FOX""",0.866252,7073
"""FALCO""",0.867229,3909
"""CAPTAIN_FALCON…",0.880541,3181
"""MARTH""",0.864424,2884
"""SHEIK""",0.837912,1820
"""PEACH""",0.855053,752
"""JIGGLYPUFF""",0.841463,410
"""DR_MARIO""",0.872,375
"""LUIGI""",0.87062,371


About what you'd expect, but gives obvious areas for improvement. Noteably lower l cancel percentage vs Jiggs, Peach, and Sheik is painful and I definitely notice it in-game

In [17]:
(
    l_cancel.filter(pl.col("character") == "FALCO")
    .groupby(pl.col("stocks_remaining"))
    .agg([pl.col("l_cancel").mean(), pl.count()])
    .sort(pl.col("stocks_remaining"), descending=True)
)


stocks_remaining,l_cancel,count
i64,f64,u32
4,0.859475,6739
3,0.866894,6461
2,0.861803,5579
1,0.875221,3967


Huh. It actually goes up as my stock count goes down. I wonder if this is an artifact from sample size, if I actually play better when I'm on my last stock, or because I play slower and more careful on my last stock. 

In [19]:
(
    l_cancel.filter(pl.col("character") == "FALCO")
    .groupby(pl.col("result"))
    .agg([pl.col("l_cancel").mean(), pl.count()])
)


result,l_cancel,count
str,f64,u32
"""loss""",0.86598,11170
"""win""",0.863856,11576


In [20]:
(
    l_cancel.filter(pl.col("character") == "FALCO")
    .groupby(pl.col("fastfall"))
    .agg([pl.col("l_cancel").mean(), pl.count()])
)

fastfall,l_cancel,count
bool,f64,u32
false,0.635144,4769
true,0.92585,17977
